In [14]:
!pip -q install --upgrade "llama-cpp-python[server]==0.2.84" google-adk litellm pandas duckdb sqlite-utils python-dateutil


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 MB 17.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 120.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.5/20.5 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3

In [15]:
import sqlite3, pandas as pd, random
from pathlib import Path
from datetime import datetime, timedelta

db_path = Path("/content/travel.db")
con = sqlite3.connect(db_path)

# Flights: simple mock inventory
flights = []
airlines = ["SkyJet","CloudAir","Pacifica","Quasar","ZenFly"]
routes = [
    ("SFO","LAX"),("SFO","JFK"),("LAX","SFO"),("SEA","SFO"),("SFO","SEA"),
    ("SFO","LAS"),("LAS","SFO"),("SJC","LAX"),("SFO","ORD"),("ORD","SFO")
]
base_date = datetime.now().date() + timedelta(days=10)

for d in range(10):
    for (orig,dest) in routes:
        depart = datetime.combine(base_date+timedelta(days=d), pd.to_datetime("08:00").time())
        arrive = depart + timedelta(hours=random.choice([1,2,5]))
        price = random.randint(90, 480)
        airline = random.choice(airlines)
        flights.append((orig,dest,depart.isoformat(),arrive.isoformat(),airline,price))

df_f = pd.DataFrame(flights, columns=["origin","destination","depart_at","arrive_at","airline","price_usd"])

# Hotels
hotels = []
cities = {
    "Los Angeles":["Silver Palm Inn","Echo Grand","Coastal Vista","Urban Hive"],
    "New York":["Hudson Square","Parkline East","NoMad Edge","Riverside 72"],
    "Las Vegas":["Neon Palace","Sands Vista","Mirage Court","Monarch Strip"],
    "Seattle":["Pine & Pike","Rain City Suites","Soundline Hotel","Fremont Gate"],
    "San Francisco":["Market Crest","Bayline Wharf","Twin Peaks Lodge","Golden Gate Stay"]
}
for city, names in cities.items():
    for name in names:
        nightly = random.randint(110, 420)
        rating = round(random.uniform(3.6, 4.9),1)
        hood = random.choice(["Downtown","Near Airport","Waterfront","Arts District","Financial"])
        hotels.append((city, name, nightly, rating, hood))

df_h = pd.DataFrame(hotels, columns=["city","hotel_name","nightly_usd","rating","neighborhood"])

df_f.to_sql("flights", con, if_exists="replace", index=False)
df_h.to_sql("hotels", con, if_exists="replace", index=False)

con.commit(); con.close()

print("DB ready at", db_path)
print("flights rows:", len(df_f), "| hotels rows:", len(df_h))


DB ready at /content/travel.db
flights rows: 100 | hotels rows: 20


In [16]:
import sqlite3, pandas as pd, json, re
from dateutil import parser as dateparser
from typing import Dict, Any, Optional, List, Tuple

DB_PATH = "/content/travel.db"

# Very small allowlist to keep SQL safe in demo
ALLOWED_TABLES = {"flights","hotels"}
ALLOWED_COLS = {
    "flights":{"origin","destination","depart_at","arrive_at","airline","price_usd"},
    "hotels":{"city","hotel_name","nightly_usd","rating","neighborhood"}
}

def _safe_cols(table: str, cols: List[str]) -> List[str]:
    ok = ALLOWED_COLS[table]
    return [c for c in cols if c in ok]

def db_query(table: str, select_cols: List[str], where: Optional[str]=None, params: Tuple=()) -> Dict[str, Any]:
    table = table.lower()
    assert table in ALLOWED_TABLES, "disallowed table"
    cols = _safe_cols(table, select_cols) or list(ALLOWED_COLS[table])
    sql = f"SELECT {', '.join(cols)} FROM {table}"
    if where:
        # naive guard: forbid semicolons and DROP/UPDATE/INSERT
        if ";" in where or re.search(r"\b(drop|update|insert|delete)\b", where, re.I):
            raise ValueError("unsafe WHERE")
        sql += f" WHERE {where}"
    con = sqlite3.connect(DB_PATH)
    df = pd.read_sql_query(sql, con, params=params)
    con.close()
    return {"table": table, "sql": sql, "rows": df.to_dict(orient="records")[:50]}

def search_flights(origin: str, destination: str, depart_date: str, max_price: Optional[int]=None) -> Dict[str, Any]:
    d0 = dateparser.parse(depart_date).date()
    d1 = d0 + timedelta(days=1)
    where = "origin = ? AND destination = ? AND depart_at >= ? AND depart_at < ?"
    params = (origin.upper(), destination.upper(), d0.isoformat(), d1.isoformat())
    res = db_query("flights", ["origin","destination","depart_at","arrive_at","airline","price_usd"], where, params)
    rows = res["rows"]
    if max_price is not None:
        rows = [r for r in rows if r["price_usd"] <= int(max_price)]
    rows.sort(key=lambda r: (r["price_usd"], r["depart_at"]))
    return {"flights": rows[:10], "query": {"origin":origin,"destination":destination,"date":str(d0),"max_price":max_price}}

def search_hotels(city: str, max_nightly: Optional[int]=None, min_rating: float=3.5) -> Dict[str, Any]:
    res = db_query("hotels", ["city","hotel_name","nightly_usd","rating","neighborhood"], "city = ?", (city.title(),))
    rows = [r for r in res["rows"] if r["rating"] >= float(min_rating)]
    if max_nightly is not None:
        rows = [r for r in rows if r["nightly_usd"] <= int(max_nightly)]
    rows.sort(key=lambda r: (-r["rating"], r["nightly_usd"]))
    return {"hotels": rows[:10], "query": {"city": city, "max_nightly": max_nightly, "min_rating": min_rating}}

def rough_budget(nights: int, hotel_nightly: int, flight_price: int, food_per_day: int=60, misc: int=120) -> Dict[str, Any]:
    total = flight_price + (hotel_nightly * nights) + (food_per_day * nights) + misc
    return {
        "nights": nights, "hotel_nightly": hotel_nightly, "flight_price": flight_price,
        "food_per_day": food_per_day, "misc": misc, "total_usd": int(total)
    }

print(search_flights("SFO","LAX", str(datetime.now().date()+timedelta(days=10)))["flights"][:1])
print(search_hotels("Los Angeles", 200)["hotels"][:1])
print(rough_budget(3, 150, 180))


[{'origin': 'SFO', 'destination': 'LAX', 'depart_at': '2025-11-01T08:00:00', 'arrive_at': '2025-11-01T10:00:00', 'airline': 'Quasar', 'price_usd': 378}]
[{'city': 'Los Angeles', 'hotel_name': 'Echo Grand', 'nightly_usd': 131, 'rating': 4.4, 'neighborhood': 'Arts District'}]
{'nights': 3, 'hotel_nightly': 150, 'flight_price': 180, 'food_per_day': 60, 'misc': 120, 'total_usd': 930}


In [17]:
from huggingface_hub import hf_hub_download
import os

REPO_ID  = "TheBloke/DeepSeek-Coder-6.7B-Instruct-GGUF"
FILENAME = "deepseek-coder-6.7b-instruct.Q4_K_M.gguf"   # ~3.8 GB

model_path = hf_hub_download(repo_id=REPO_ID, filename=FILENAME)
os.environ["MODEL_PATH"] = model_path
print("MODEL_PATH =", os.environ["MODEL_PATH"])


MODEL_PATH = /root/.cache/huggingface/hub/models--TheBloke--DeepSeek-Coder-6.7B-Instruct-GGUF/snapshots/9e221e6b41cb1bf1c5d8f9718e81e3dc781f7557/deepseek-coder-6.7b-instruct.Q4_K_M.gguf


In [18]:
%%bash
# 3.A1 — start server in background and log to /content/server.log
pkill -f "llama_cpp.server.*--port 8000" || true

nohup python -m llama_cpp.server \
  --model "$MODEL_PATH" \
  --host 127.0.0.1 --port 8000 \
  --model_alias deepseek-local \
  --chat_format chatml \
  --n_ctx 4096 \
  > /content/server.log 2>&1 &

sleep 3
tail -n 50 /content/server.log


llm_load_vocab: token to piece cache size = 0.1795 MB
llm_load_print_meta: format           = GGUF V3 (latest)
llm_load_print_meta: arch             = llama
llm_load_print_meta: vocab type       = BPE
llm_load_print_meta: n_vocab          = 32256
llm_load_print_meta: n_merges         = 31757
llm_load_print_meta: vocab_only       = 0
llm_load_print_meta: n_ctx_train      = 16384
llm_load_print_meta: n_embd           = 4096
llm_load_print_meta: n_layer          = 32
llm_load_print_meta: n_head           = 32
llm_load_print_meta: n_head_kv        = 32
llm_load_print_meta: n_rot            = 128
llm_load_print_meta: n_swa            = 0
llm_load_print_meta: n_embd_head_k    = 128
llm_load_print_meta: n_embd_head_v    = 128
llm_load_print_meta: n_gqa            = 1
llm_load_print_meta: n_embd_k_gqa     = 4096
llm_load_print_meta: n_embd_v_gqa     = 4096
llm_load_print_meta: f_norm_eps       = 0.0e+00
llm_load_print_meta: f_norm_rms_eps   = 1.0e-06
llm_load_print_meta: f_clamp_kqv      = 0.0

In [19]:
# 3.A2 — wait for server to be ready
import time, requests, subprocess

base = "http://127.0.0.1:8000"
for i in range(120):  # up to ~6 minutes on CPU
    try:
        r = requests.get(f"{base}/v1/models", timeout=2)
        if r.ok:
            print("Server is ready ✅")
            print(r.json())
            break
    except Exception:
        pass
    if i % 5 == 0:
        print(f"...waiting ({i*3}s)")
    time.sleep(3)
else:
    print("Server not ready; last 100 log lines:")
    subprocess.run(["tail","-n","100","/content/server.log"])


...waiting (0s)
...waiting (15s)
Server is ready ✅
{'object': 'list', 'data': [{'id': 'deepseek-local', 'object': 'model', 'owned_by': 'me', 'permissions': []}]}


In [20]:
import os, requests, json

# Point to your local llama.cpp server
os.environ["OPENAI_API_BASE"] = "http://127.0.0.1:8000/v1"
os.environ["OPENAI_API_KEY"]  = "sk-local-anything"   # dummy value satisfies the client

# Optional: shorter timeouts so it doesn't hang forever
os.environ["LITELLM_TIMEOUT"] = "120"
os.environ["LITELLM_REQUEST_TIMEOUT"] = "120"
os.environ["LITELLM_MAX_RETRIES"] = "0"

# Sanity check: server responding?
r = requests.get("http://127.0.0.1:8000/v1/models", timeout=5)
print("Models:", r.json())


Models: {'object': 'list', 'data': [{'id': 'deepseek-local', 'object': 'model', 'owned_by': 'me', 'permissions': []}]}


In [22]:
# === ADK Travel Agent (DeepSeek) — city<->IATA mapping + robust fallback ===
import json
from typing import List, Optional
from datetime import datetime, timedelta

# ADK bits
from google.adk.agents import LlmAgent
from google.adk.models.lite_llm import LiteLlm
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types

# ---- City <-> IATA helpers ----
CITY_TO_IATA = {
    "san francisco": "SFO",
    "los angeles": "LAX",
    "seattle": "SEA",
    "las vegas": "LAS",
    "chicago": "ORD",
    "new york": "JFK",
    "sjc": "SJC",
}
IATA_TO_CITY = {v: k.title() for k, v in CITY_TO_IATA.items()}

def to_iata(text: str) -> str:
    if not text: return ""
    t = text.strip().upper()
    if len(t) == 3 and t.isalpha():
        return t  # already an IATA
    return CITY_TO_IATA.get(text.strip().lower(), t)

def to_city(text: str) -> str:
    if not text: return ""
    t = text.strip().upper()
    if len(t) == 3 and t in IATA_TO_CITY:
        return IATA_TO_CITY[t]
    return text.strip().title()

# ---- tiny selectors (keep your behavior) ----
def choose_best_flight(flights: List[dict]) -> Optional[dict]:
    return flights[0] if flights else None

def choose_best_hotel(hotels: List[dict]) -> Optional[dict]:
    return hotels[0] if hotels else None

# ---- Composer agent (DeepSeek local via LiteLLM) ----
composer = LlmAgent(
    model=LiteLlm(model="openai/deepseek-local"),
    name="travel_itinerary_composer",
    description="Writes a concise travel plan from given tool outputs.",
    instruction=(
        "You are a helpful travel planner. Produce a concise Markdown itinerary with:\n"
        "1) Trip Summary; 2) Flights (chosen option + 2 alternates if available);\n"
        "3) Hotel (chosen option + 2 alternates if available);\n"
        "4) Daily Plan (bullets per day); 5) Budget Summary using provided numbers; 6) Tips.\n"
        "Use ONLY the numbers provided. Keep it to ~300–400 words and make it practical."
    ),
    tools=[],
)

APP_NAME="adk_travel_demo"
USER_ID="student"
SESSION_ID="session-travel-001"

# NOTE: This function assumes you've already defined:
#   - search_flights(origin_iata, dest_iata, depart_date, max_price)
#   - search_hotels(city, max_nightly, min_rating)
#   - rough_budget(nights, hotel_nightly, flight_price, food_per_day=60, misc=120)

async def plan_trip(origin:str, destination:str, depart_date:str, nights:int,
                    max_flight:int=400, max_hotel:int=220) -> str:
    """
    origin/destination can be city names or IATA codes.
    We normalize: flights use IATA, hotels use city name.
    """
    origin_iata = to_iata(origin)
    dest_iata   = to_iata(destination)
    dest_city   = to_city(destination)

    # 1) DB-backed tools
    f = search_flights(origin_iata, dest_iata, depart_date, max_price=max_flight)
    h = search_hotels(dest_city, max_nightly=max_hotel, min_rating=3.8)

    flights_top = f["flights"][:3]
    hotels_top  = h["hotels"][:3]
    pick_f = choose_best_flight(flights_top)
    pick_h = choose_best_hotel(hotels_top)

    budget = (rough_budget(nights, pick_h["nightly_usd"], pick_f["price_usd"])
              if (pick_f and pick_h) else {"total_usd": None})

    # 2) Compose prompt pack (compact)
    pack = {
        "inputs": {
            "origin": origin_iata, "destination": dest_iata, "destination_city": dest_city,
            "depart_date": depart_date, "nights": nights,
            "caps": {"max_flight": max_flight, "max_hotel": max_hotel},
        },
        "flights_top": flights_top,
        "hotels_top": hotels_top,
        "chosen": {"flight": pick_f, "hotel": pick_h},
        "budget": budget,
    }

    user_msg = (
        "Write the final Markdown itinerary from these tool outputs. Use only provided numbers.\n\n"
        f"{json.dumps(pack, indent=2)}\n\n"
        "Sections: 1) Trip Summary 2) Flights 3) Hotel 4) Daily Plan 5) Budget Summary 6) Tips.\n"
        "Be concise and specific."
    )

    # 3) ADK compose
    session_service = InMemorySessionService()
    await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
    runner = Runner(agent=composer, app_name=APP_NAME, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=user_msg)])

    final_text = None
    acc = ""
    try:
        for event in runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
            if getattr(event, "content", None) and getattr(event.content, "parts", None):
                t = getattr(event.content.parts[0], "text", None)
                if t:
                    acc += t
                    final_text = acc
                    if len(acc) > 2000:  # ~350–400 words
                        break
    except Exception as e:
        print("[LLM error] Falling back due to:", repr(e))
        final_text = None

    # 4) Fallback if DeepSeek refuses or error
    bad = (not final_text) or any(x in final_text.lower() for x in ["i'm sorry","as an ai","cannot","unable"])
    if bad:
        lines = [f"# Travel Itinerary — {to_city(origin_iata)} → {dest_city}",
                 f"**Dates:** depart {depart_date}, {nights} nights"]
        if pick_f:
            lines.append(
                f"**Flight (chosen):** {pick_f['airline']} {pick_f['origin']}→{pick_f['destination']} "
                f"{pick_f['depart_at']} → {pick_f['arrive_at']} — **${pick_f['price_usd']}**"
            )
            if len(flights_top) > 1:
                lines.append("**Alternates:**")
                for alt in flights_top[1:]:
                    lines.append(
                        f"- {alt['airline']} {alt['origin']}→{alt['destination']} "
                        f"{alt['depart_at']} → {alt['arrive_at']} — ${alt['price_usd']}"
                    )
        if pick_h:
            lines.append(
                f"**Hotel (chosen):** {pick_h['hotel_name']} ({pick_h['neighborhood']}), "
                f"rating {pick_h['rating']}, **${pick_h['nightly_usd']}/night**"
            )
            if len(hotels_top) > 1:
                lines.append("**Hotel Alternates:**")
                for alt in hotels_top[1:]:
                    lines.append(
                        f"- {alt['hotel_name']} ({alt['neighborhood']}), rating {alt['rating']}, ${alt['nightly_usd']}/night"
                    )
        if budget.get("total_usd"):
            lines.append(f"**Budget (rough):** ≈ **${budget['total_usd']}** total for {nights} nights")
        lines += [
            "## Daily Plan",
            "- Day 1: Arrival, check-in, short walk + casual dinner.",
            "- Day 2: Morning highlight, afternoon museum/park, evening district.",
            "- Day 3+: Flexible; local markets, viewpoints, food.",
            "## Tips",
            "- Book refundable fares when possible.",
            "- Consider off-peak attractions to avoid lines.",
        ]
        final_text = "\n\n".join(lines)

    return final_text

# ---- One run (you can pass 'Los Angeles' or 'LAX') ----
itinerary_md = await plan_trip(
    origin="SFO",
    destination="Los Angeles",
    depart_date=str((datetime.now().date()+timedelta(days=10))),
    nights=3,
    max_flight=250,
    max_hotel=220
)
print(itinerary_md[:1500])


# Travel Itinerary — San Francisco → Los Angeles

**Dates:** depart 2025-11-02, 3 nights

**Flight (chosen):** ZenFly SFO→LAX 2025-11-02T08:00:00 → 2025-11-02T10:00:00 — **$231**

**Hotel (chosen):** Echo Grand (Arts District), rating 4.4, **$131/night**

**Hotel Alternates:**

- Urban Hive (Financial), rating 3.9, $198/night

**Budget (rough):** ≈ **$924** total for 3 nights

## Daily Plan

- Day 1: Arrival, check-in, short walk + casual dinner.

- Day 2: Morning highlight, afternoon museum/park, evening district.

- Day 3+: Flexible; local markets, viewpoints, food.

## Tips

- Book refundable fares when possible.

- Consider off-peak attractions to avoid lines.
